# Dataset Preparation

## Imports

In [ ]:
import os, glob, re
import pandas as pd
from sklearn.model_selection import train_test_split

## Constants

In [ ]:
# constants
MIN_LENGTH = 80

DATASET_PREFIX = '../dataset/'
ENGLISH_PATH = DATASET_PREFIX + '1_english/'
URLS_MENTIONS_REMOVED_PATH = DATASET_PREFIX + '2_filtered_urls_mentions/'
MIN_LENGTH_PATH = DATASET_PREFIX + '3_length_greater_than_{0}/'.format(MIN_LENGTH)
STOPWORDS_PATH = DATASET_PREFIX + '4_no-stopword/'
NONROMAN_PATH = DATASET_PREFIX + '5_non-roman/'
PREPROCESSED_PATH = DATASET_PREFIX + '6_pre-processed/'
PROCESSED_PATH = DATASET_PREFIX + '7_processed/'

## Load Data

In [ ]:
def get_file_pairs(files_dir):
    files = glob.glob(files_dir + '*.csv')
    file_pairs = []

    for each in files:
        file_name = each.split('/')[-1:][0]

        dF = pd.read_csv(each, usecols=['Text'])
        dF['Text'] = dF['Text'].astype('str')
        file_pairs.append((file_name, dF))
        print('filename: {0}'.format(each))
        print('size:     {0}'.format(dF.size))
        print('----------------------------------------------------\n')
    
    return file_pairs

## Filter out URLs and @ Mentions

In [ ]:
english_file_pairs = get_file_pairs(ENGLISH_PATH)

In [ ]:
# URLS_REMOVED_PATH
regex_urls = r'(https?://.+\b\/?|.{3}\.twitter.+\b)'
regex_mentions = r'@\w+'
regex = re.compile(regex_urls)

if not os.path.exists(URLS_MENTIONS_REMOVED_PATH):
    os.makedirs(URLS_MENTIONS_REMOVED_PATH)

for each in english_file_pairs:
    filename = each[0]
    dF = each[1].copy()
    output_file = URLS_MENTIONS_REMOVED_PATH + filename

    # remove URLs
    dF.Text = dF.apply(lambda x: x.str.replace(regex_urls, ' '))
    # remove mentions
    dF.Text = dF.apply(lambda x: x.str.replace(regex_mentions, ' '))
    # save
    dF.to_csv(output_file, index=False, columns=['Text'])
    print(dF, '\n----------------------------------------------------')    

## Filter for MIN_LENGTH

In [ ]:
url_filtered_file_pairs = get_file_pairs(URLS_MENTIONS_REMOVED_PATH)

In [ ]:
if not os.path.exists(MIN_LENGTH_PATH):
    os.makedirs(MIN_LENGTH_PATH)


for each in url_filtered_file_pairs:
    filename = each[0]
    dF = each[1].copy()
    output_file = MIN_LENGTH_PATH + filename
    print(output_file)

    dF = dF[ dF['Text'].str.len() > MIN_LENGTH]
    
    # save
    dF.to_csv(output_file, index=False, columns=['Text'])

    print(dF, '\n----------------------------------------------------')

## Stopword Removal

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')

In [ ]:
# min_length_file_pairs = get_file_pairs(MIN_LENGTH_PATH)

In [ ]:
# if not os.path.exists(STOPWORDS_PATH):
#     os.makedirs(STOPWORDS_PATH)

# STOP_WORDS = stopwords.words('english')
# print('STOP_WORDS', STOP_WORDS)
    
# def remove_stopwords(texts):
#     new_texts = []
#     for tx in texts:
#         splitted = tx.lower().split(' ')
#         removed = [word for word in splitted if not word in STOP_WORDS]
#         removed = ' '.join(removed)
#         new_texts.append(removed)
#     return new_texts

# for each in min_length_file_pairs:
#     filename = each[0]
#     dF = each[1].copy()
#     output_file = STOPWORDS_PATH + filename
#     print('output_file', output_file)

#     # remove all stop_words
#     dF.Text = remove_stopwords(dF.Text)

#     # save
#     dF.to_csv(output_file, index=False, columns=['Text'])

#     print(dF, '\n----------------------------------------------------')

## Roman English Removal
> Removes roman english tweets

In [ ]:
from enchant.checker import SpellChecker

In [ ]:
minlength_file_pairs = get_file_pairs(MIN_LENGTH_PATH)

In [ ]:
if not os.path.exists(NONROMAN_PATH):
    os.makedirs(NONROMAN_PATH)


ROMAN_THRESHOLD = .5

def remove_romans(texts):
    """
    Check if 50% of each sentence has valid english words
    """
    new_texts = []
    d = SpellChecker("en_US")
    
    for quote in texts:      
        d.set_text(quote)
        errors = [err.word for err in d]
        target_length = len(quote.split()) * ROMAN_THRESHOLD
        if len(errors) <= target_length:
            new_texts.append(quote)

    return pd.DataFrame({'Text': new_texts})

for each in minlength_file_pairs:
    filename = each[0]
    dF = each[1].copy()
    output_file = NONROMAN_PATH + filename
    print('output_file', output_file)

    # filter sentences that have more than 50% non-english content
    dF = remove_romans(dF.Text)

    # save
    dF.to_csv(output_file, index=False, columns=['Text'])

    print(dF, '\n----------------------------------------------------')

## Prepare Dataset
- change case to smaller-case
- remove punctuation
- replace multiple consecutive spaces with single space
- filter for MIN_LENGTH
- add class labels

In [ ]:
file_pairs = get_file_pairs(NONROMAN_PATH)

In [ ]:
if not os.path.exists(PREPROCESSED_PATH):
    os.makedirs(PREPROCESSED_PATH)

for each in file_pairs:
    file_name = each[0]
    label = file_name.split('.csv')[0]
    dF = each[1].copy()
    output_file = PREPROCESSED_PATH + file_name

    # convert to lower-case
    dF.Text = dF.apply(lambda x: x.str.lower())
    # remove punctuation
    dF.Text = dF.apply(lambda x: x.str.replace(r"[^a-zA-Z_\s0-9#]", ''))
    # remove extra spaces
    dF.Text = dF.apply(lambda x: x.str.replace(r'\s{2,}', ' '))
    # filter for len(tweets) > MIN_LENGTH x 2 to get more meaningful tweets
    dF = dF[ dF['Text'].str.len() > MIN_LENGTH]

    # add label
    dF['Label'] = label

    dF.to_csv(output_file, index=False, columns=['Label','Text'])
    
    print(dF)

### Create Dataset Files
- single dataset file
- split to train, test, validation sets

#### dataset.csv

In [ ]:
def get_prefixed_file_pairs(files_dir):
    files = glob.glob(files_dir + '*.csv')
    file_pairs = []

    for each in files:
        file_name = each.split('/')[-1:][0]

        dF = pd.read_csv(each, usecols=['Label', 'Text'])
        dF['Text'] = dF['Text'].astype('str')
        file_pairs.append((file_name, dF))
        print('filename: {0}'.format(each))
        print('size:     {0}'.format(len(dF)))
        print('----------------------------------------------------\n')
    
    return file_pairs

In [ ]:
file_pairs = get_prefixed_file_pairs(PREPROCESSED_PATH)

In [ ]:
frames_list = []
for each in file_pairs:
    dF = each[1].copy()
    frames_list.append(dF)

bigDF = pd.concat(frames_list)

if not os.path.exists(PROCESSED_PATH):
    os.makedirs(PROCESSED_PATH)

bigDF.to_csv(PROCESSED_PATH + 'dataset.csv', index=False, columns=['Label', 'Text'])

print(bigDF)

#### Train, Test, Val Split

In [ ]:
input_file = PROCESSED_PATH + 'dataset.csv';
datasetFrame = pd.read_csv(input_file)
print('dataset length:', len(datasetFrame))

In [ ]:
non_test_set, test_set = train_test_split(datasetFrame, test_size=0.2, random_state=18030010)
train_set, val_set = train_test_split(non_test_set, test_size=0.15, random_state=18030010)

len(train_set), len(val_set), len(test_set)

In [ ]:
train_set.to_csv(PROCESSED_PATH + 'train.csv', index=False, columns=['Label', 'Text'])
print('Train Set\n',train_set)

val_set.to_csv(PROCESSED_PATH + 'val.csv', index=False, columns=['Label', 'Text'])
print('Val Set\n',val_set)

test_set.to_csv(PROCESSED_PATH + 'test.csv', index=False, columns=['Label', 'Text'])
print('Test Set\n',test_set)

## Final Count

In [ ]:
datasetFrame.Label.value_counts().sort_values(ascending=False).plot(kind='bar', y='# of tweets', title='# of tweets per politician') 

## Fasttext Modifications

In [ ]:
FASTTEXT_PATH = DATASET_PREFIX + '8_fasttext/'

if not os.path.exists(FASTTEXT_PATH):
    os.makedirs(FASTTEXT_PATH)

TRAIN_FILE = PROCESSED_PATH + 'train.csv'
VALDN_FILE = PROCESSED_PATH + 'val.csv'
TEST_FILE  = PROCESSED_PATH + 'test.csv'

trainFrame = pd.read_csv(TRAIN_FILE)
valdnFrame = pd.read_csv(VALDN_FILE)
testFrame  = pd.read_csv(TEST_FILE)

print('dataset length:', len(trainFrame), len(valdnFrame), len(testFrame))

In [ ]:
def modify_for_fasttext(dF):
    # add __label__ prefix with labels
    dF.Label = '__label__' + dF.Label
    return dF

In [ ]:
trnFrame = modify_for_fasttext(trainFrame.copy())
valFrame = modify_for_fasttext(valdnFrame.copy())
tstFrame = modify_for_fasttext(testFrame.copy())

In [ ]:
trnFrame.to_csv(FASTTEXT_PATH + 'train.csv', index=False, sep=' ', 
                header=False, columns=['Label','Text'])
valFrame.to_csv(FASTTEXT_PATH + 'val.csv', index=False, sep=' ', 
                header=False, columns=['Label','Text'])
tstFrame.to_csv(FASTTEXT_PATH + 'test.csv', index=False, sep=' ', 
                header=False, columns=['Label','Text'])